# Preprocessing

* **After applying initial EDA to our datasets, We'll now target the following preprocessing tasks:**

    * Handle null values and inconsistencies for all features.
    * Create a target variable (issue_happened) reflecting whether
    a problem happened during the transfer of this package/receptacle.
    * Propose, validate assumptions (with Ms.Lasmi), and work based on them.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
packages_df = pd.read_csv('../data/raw/packages_data_2023_2025.csv', encoding='latin-1', delimiter=';')
receptacles_df = pd.read_csv('../data/raw/receptacle_data_2023_2025.csv', encoding='latin-1', delimiter=';')

* Columns Renaming

In [ ]:
packages_df = packages_df.rename(columns={'établissement_postal': 'etablissement_postal', 'next_établissement_postal': 'next_etablissement_postal'})
receptacles_df = receptacles_df.rename(columns={'ï»¿RECPTCL_FID': 'RECPTCL_FID', 'EVENT_TYPECD': 'EVENT_TYPE_CD', 'nextetablissement_postal': 'next_etablissement_postal'})

* `date` type adjustment

In [ ]:
packages_df['date'] = pd.to_datetime(packages_df['date'])
receptacles_df['date'] = pd.to_datetime(receptacles_df['date'])
packages_df['RECPTCL_FID'] = packages_df['RECPTCL_FID'].str.strip()
packages_df['MAILITM_FID'] = packages_df['MAILITM_FID'].str.strip()
packages_df['etablissement_postal'] = packages_df['etablissement_postal'].str.strip()
packages_df['next_etablissement_postal'] = packages_df['next_etablissement_postal'].str.strip()
receptacles_df['etablissement_postal'] = receptacles_df['etablissement_postal'].str.strip()
receptacles_df['next_etablissement_postal'] = receptacles_df['next_etablissement_postal'].str.strip()
receptacles_df['RECPTCL_FID'] = receptacles_df['RECPTCL_FID'].str.strip()

- split into train and test

didn't use time series split since the split would be done into folds instead of one train test split 
even if in the future the time series split could be more usefull but for now it is just a complication 

In [ ]:
packages_df = packages_df.sort_values('date').reset_index(drop=True)
receptacles_df = receptacles_df.sort_values('date').reset_index(drop=True)

In [ ]:
# Time-respecting split
pkg_train_df, pkg_test_df = train_test_split(
    packages_df,
    test_size=0.2,
    shuffle=False
)

rcl_train_df, rcl_test_df = train_test_split(
    receptacles_df,
    test_size=0.2,
    shuffle=False
)

In [ ]:
packages_df = pkg_train_df
receptacles_df= rcl_train_df


In [ ]:
packages_df.head()

In [ ]:
packages_df.info()

In [ ]:
receptacles_df.head()

In [ ]:
receptacles_df.info()

In [ ]:
packages_df['etablissement_postal'].isna().sum()

* `etablissement_postal` have 26621 null values (2.7% of the whole dataset)

* As its null values are less than 5% of the dataset (2.7%), we drop these null values

In [ ]:
packages_df = packages_df[~packages_df['etablissement_postal'].isna()]
packages_df['etablissement_postal'].isna().sum()

* We propose to consider the packages having null `next_etablissement_postal`
as having issue during transfer, we'll try to validate that using
`EVENT_TYPE_CD` also

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# packages_df['next_etablissement_postal'] = packages_df['next_etablissement_postal'].fillna('Unknown')
# packages_df['next_etablissement_postal'].isna().any()

* Let's check if `EVENT_TYPE_CD` can indicate whether the `next_etablissement_postal` is null or not

In [ ]:
packages_unknown_next_etablissement = packages_df[packages_df['next_etablissement_postal'].isna()]
# keep only top EVENT_TYPES_ID
packages_unknown_next_etablissement = packages_unknown_next_etablissement['EVENT_TYPE_CD'].value_counts()

plt.figure(figsize=(12, 6))
packages_unknown_next_etablissement.head(10).plot(kind='bar')
plt.xlabel('EVENT TYPE CD')
plt.ylabel('Null Next Etablissement Postal')
plt.xticks(rotation=0)
plt.show()

* `EVENT_TYPE_CD` doesn't indicate null values of `next_etablissement_postal`

* the following code cell fills the `next_etablissement_postal` using the next `etablissement_postal` for the same package.
* if the last route for a specific package is null, then it keeps it null because there's no next `etablissement_postal` for that package.

In [ ]:
# 1. Ensure the dataframe is sorted (same as before)
packages_df = packages_df.sort_values(['MAILITM_FID', 'date'])

# 2. Look ahead to the next row's postal code and ID
shifted_postal = packages_df['etablissement_postal'].shift(-1)
shifted_id = packages_df['MAILITM_FID'].shift(-1)

# 3. Identify the "boundaries" where the postal code changes within the same package
# This marks the last row of a block with the value of the start of the next block
is_boundary = (packages_df['etablissement_postal'] != shifted_postal) & \
              (packages_df['MAILITM_FID'] == shifted_id)

# 4. Use grouped backfill to broadcast those values to all preceding rows in the block
# This replaces your 'blocks.map' logic with a single vectorized pass
fill_values = shifted_postal.where(is_boundary).groupby(packages_df['MAILITM_FID']).bfill()

# 5. Fill only the NaNs in the existing column to match your original logic
packages_df['next_etablissement_postal'] = packages_df['next_etablissement_postal'].fillna(fill_values)

In [ ]:
# 1. Ensure the dataframe is sorted (same as before)
packages_df = packages_df.sort_values(['MAILITM_FID', 'date'])

# 2. Look ahead to the next row's postal code and ID
shifted_postal = packages_df['etablissement_postal'].shift(-1)
shifted_id = packages_df['MAILITM_FID'].shift(-1)

# 3. Identify the "boundaries" where the postal code changes within the same package
# This marks the last row of a block with the value of the start of the next block
is_boundary = (packages_df['etablissement_postal'] != shifted_postal) & \
              (packages_df['MAILITM_FID'] == shifted_id)

# 4. Use grouped backfill to broadcast those values to all preceding rows in the block
# This replaces your 'blocks.map' logic with a single vectorized pass
fill_values = shifted_postal.where(is_boundary).groupby(packages_df['MAILITM_FID']).bfill()

# 5. Fill only the NaNs in the existing column to match your original logic
packages_df['next_etablissement_postal'] = packages_df['next_etablissement_postal'].fillna(fill_values)

In [ ]:
packages_df.head()

In [ ]:
packages_df_copy = packages_df.copy()
# make a copy to prevent executing the cell
# above many times (it takes too long)

In [ ]:
packages_df_copy.info()

* To handle those null values of `next_etablissement_postal`, we'll try to predict them based on the `etablissement_postal` they appear with more frequently.

In [ ]:
etablissement_dict = {} # will hold each etablissement_postal
# and the next_etablissement_postal that appeared the most 
# with it.
for etablissement, group in packages_df_copy.groupby('etablissement_postal'):
    next_counts = group['next_etablissement_postal'].dropna().value_counts()
    if next_counts.empty:
        # no observed next_etablissement_postal for this etablissement; skip or fallback
        # use predefined next_etablissement if available, else skip
        fallback = globals().get('next_etablissement', None)
        if fallback is not None:
            etablissement_dict[etablissement] = fallback
        continue
    etablissement_dict[etablissement] = next_counts.index[0]

etablissement_dict

In [ ]:
len(etablissement_dict.keys()), packages_df_copy['etablissement_postal'].nunique()

In [ ]:
# Create mask
null_mask = packages_df_copy['next_etablissement_postal'].isna()

# Apply to original dataframe directly
packages_df_copy.loc[null_mask, 'next_etablissement_postal'] = (
    packages_df_copy.loc[null_mask, 'etablissement_postal'].map(etablissement_dict)
)

packages_df_copy.head()

In [ ]:
packages_df_copy.info()

In [ ]:
packages_df_copy['MAILITM_FID'].nunique()

* Like this, we've handled all null values and inconsitencies for `packages` dataset, and we can start applying tasks further.
* Now, I propose to encode the following categorical features `etablissement_postal`, `next_etablissement_postal`, `origin_hub`, `destination_country`, `arrival_hub`, `inbound_unit`

* If we Encode the following features `origin_hub`, `arrival_hub`, `inbound_unit`, and `destination_country` with one-hot encoding, we'd have more than 100 features, so I'm just gonna keep them categorical for now.

* **We'll be doing the same steps for `receptacle` dataset**

In [ ]:
receptacles_df.head()

In [ ]:
receptacles_df = receptacles_df[~receptacles_df['etablissement_postal'].isna()]

* apply the function that fills null values of `next_etablissement_postal` using `etablissement_postal`

In [ ]:
# 1. Ensure sorted order
receptacles_df = receptacles_df.sort_values(['RECPTCL_FID', 'date'])

# 2. Identify transitions where the postal code changes within the same receptacle
# This finds the "start of the next block"
next_postal = receptacles_df['etablissement_postal'].shift(-1)
next_id = receptacles_df['RECPTCL_FID'].shift(-1)

# Logic: If current postal != next postal AND we are still in the same ID, 
# then 'next_postal' is the value of the next block.
is_boundary = (receptacles_df['etablissement_postal'] != next_postal) & \
              (receptacles_df['RECPTCL_FID'] == next_id)

# 3. Create a series that only contains values at those boundaries
boundary_values = next_postal.where(is_boundary)

# 4. Use grouped backfill (bfill) to propagate the next block's source 
# to all rows in the current block
filled_values = boundary_values.groupby(receptacles_df['RECPTCL_FID']).bfill()

# 5. Fill NaNs in the target column as per your original logic
receptacles_df['next_etablissement_postal'] = receptacles_df['next_etablissement_postal'].fillna(filled_values)

In [ ]:
receptacles_df.head()

In [ ]:
receptacles_df.info()

In [ ]:
receptacles_df_copy = receptacles_df.copy()

* fill remaining null values of `next_etablissement_postal` with the most frequent value appearing with its `etablissement_postal`.

In [ ]:
rcp_etablissement_dict = {} # will hold each etablissement_postal
# and the next_etablissement_postal that appeared the most 
# with it.
for etablissement, group in receptacles_df_copy.groupby('etablissement_postal'):
    next_counts = group['next_etablissement_postal'].dropna().value_counts()
    if next_counts.empty:
        # no observed next_etablissement_postal for this etablissement; skip or fallback
        # use predefined next_etablissement if available, else skip
        fallback = globals().get('next_etablissement', None)
        if fallback is not None:
            rcp_etablissement_dict[etablissement] = fallback
        continue
    rcp_etablissement_dict[etablissement] = next_counts.index[0]

rcp_etablissement_dict

In [ ]:
# Create mask
null_mask = packages_df_copy['next_etablissement_postal'].isna()

# Apply to original dataframe directly
packages_df_copy.loc[null_mask, 'next_etablissement_postal'] = (
    packages_df_copy.loc[null_mask, 'etablissement_postal'].map(etablissement_dict)
)

packages_df_copy.head()

In [ ]:
receptacles_df_copy.info()

* Null values are gone, but there are still some illogical packages' and receptacles' routes between `etablissement`
* We'll treat these logical routes now

In [ ]:
# for each package (group of rows), check whether there's any illogical route
# between 'etablissement_postal' and 'next_etablissement_postal'
def isPackageIllogical(group):
    return (
        group['next_etablissement_postal']
        .iloc[:-1]
        .ne(group['etablissement_postal'].shift(-1).iloc[:-1])
        .any()
    )

In [ ]:
illogical_packages = packages_df_copy.groupby('MAILITM_FID').apply(isPackageIllogical)
illogical_packages.value_counts()

In [ ]:
105262 / packages_df_copy['MAILITM_FID'].nunique()

* 103376 Packages have illogical routes (98%) of all packages
* as a lot of packages have at least one illogical route, we'll create three datasets and keep the one that gives the best values
* 1st dataset `clean` -> drop all packages having at leeast one illogical route
* 2nd dataset `slightly` -> drop all packages having more than one illogical route (we tolerate one illogical route)
* 3rd dataset `chaotic` -> the current dataset.
* For the last two datasets, we add a feature `illogical` to flag illogical routes, it may be useful later.

In [ ]:
clean_packages_df = packages_df_copy[~packages_df_copy['MAILITM_FID'].isin(illogical_packages[illogical_packages].index)]
clean_packages_df['MAILITM_FID'].nunique()

In [ ]:
clean_packages_df['MAILITM_FID'].nunique()

* Creating `slightly`:

In [ ]:
# 1. Standardize IDs in the source dataframe first
packages_df_copy['MAILITM_FID'] = packages_df_copy['MAILITM_FID'].astype(str).str.strip().str.upper()

# 2. Vectorized calculation (Logic equivalent to your iloc[:-1] comparison)
# We look at the next row's postal code and check if the package ID is the same
shifted_postal = packages_df_copy['etablissement_postal'].shift(-1)
is_same_package = packages_df_copy['MAILITM_FID'] == packages_df_copy['MAILITM_FID'].shift(-1)

# A mismatch counts if the next row is the same package but has a different postal code
mismatches = (packages_df_copy['next_etablissement_postal'] != shifted_postal) & is_same_package

# 3. Sum the mismatches per group (this produces a unique ID list)
illogical_counts = (
    mismatches.groupby(packages_df_copy['MAILITM_FID'], sort=False)
    .sum()
    .astype(int)
    .reset_index(name='n_illogical_routes')
)

illogical_counts

In [ ]:
illogical_counts['MAILITM_FID'] = illogical_counts['MAILITM_FID'].str.strip().str.upper()
illogical_counts[illogical_counts['n_illogical_routes'] > 1].count()

In [ ]:
illogical_counts[illogical_counts['n_illogical_routes'] <= 1].count()

In [ ]:
illogical_counts[illogical_counts['MAILITM_FID'].duplicated()]

In [ ]:
multiple_illogical_routes_packages = illogical_counts[illogical_counts['n_illogical_routes'] > 1]['MAILITM_FID'].unique()
print (len(multiple_illogical_routes_packages))
slightly_packages_df = packages_df_copy[~packages_df_copy['MAILITM_FID'].isin(multiple_illogical_routes_packages)]
slightly_packages_df['MAILITM_FID'].nunique()

In [ ]:
# Verify sums
print(f"Total packages: {packages_df_copy['MAILITM_FID'].nunique()}")
print(f"Clean (0 illogical): {clean_packages_df['MAILITM_FID'].nunique()}")
print(f"Slightly (0 or 1 illogical): {slightly_packages_df['MAILITM_FID'].nunique()}")
print(f"More than 1 illogical: {len(multiple_illogical_routes_packages)}")
print(f"\nSlightly + Multiple = {slightly_packages_df['MAILITM_FID'].nunique() + len(multiple_illogical_routes_packages)}")
print(f"Should equal total: {packages_df_copy['MAILITM_FID'].nunique()}")

* Storing datasets

In [ ]:
clean_packages_df.to_csv('../data/interim/clean_packages_df.csv')
slightly_packages_df.to_csv('../data/interim/slightly_packages_df.csv')
packages_df_copy.to_csv('../data/interim/chaotic_packages_df.csv')

### Handling Illogical Routes for receptacles (same logic with packages)

In [ ]:
# Create mask
null_mask = receptacles_df_copy['next_etablissement_postal'].isna()

# Apply to original dataframe directly
receptacles_df_copy.loc[null_mask, 'next_etablissement_postal'] = (
    receptacles_df_copy.loc[null_mask, 'etablissement_postal'].map(rcp_etablissement_dict)
)

receptacles_df_copy.head()

In [ ]:
receptacles_df_copy.info()

In [ ]:
# for each receptacle (group of rows), check whether there's any illogical route
# between 'etablissement_postal' and 'next_etablissement_postal'
def isReceptacleIllogical(group):
    return (
        group['next_etablissement_postal']
        .iloc[:-1]
        .ne(group['etablissement_postal'].shift(-1).iloc[:-1])
        .any()
    )

In [ ]:
illogical_receptacles = receptacles_df_copy.groupby('RECPTCL_FID').apply(isReceptacleIllogical)
illogical_receptacles.value_counts()

In [ ]:
# number of illogical receptacles on the total number of receptacles

In [ ]:
clean_receptacles_df = receptacles_df_copy[~receptacles_df_copy['RECPTCL_FID'].isin(illogical_receptacles[illogical_receptacles].index)]
clean_receptacles_df['RECPTCL_FID'].nunique()

In [ ]:
# 1. Standardize IDs in the source dataframe first
receptacles_df_copy['RECPTCL_FID'] = receptacles_df_copy['RECPTCL_FID'].astype(str).str.strip().str.upper()

# 2. Vectorized calculation (Logic equivalent to your iloc[:-1] comparison)
# We look at the next row's postal code and check if the receptacle ID is the same
shifted_postal = receptacles_df_copy['etablissement_postal'].shift(-1)
is_same_receptacle = receptacles_df_copy['RECPTCL_FID'] == receptacles_df_copy['RECPTCL_FID'].shift(-1)

# A mismatch counts if the next row is the same package but has a different postal code
mismatches = (receptacles_df_copy['next_etablissement_postal'] != shifted_postal) & is_same_receptacle

# 3. Sum the mismatches per group (this produces a unique ID list)
rcp_illogical_counts = (
    mismatches.groupby(receptacles_df_copy['RECPTCL_FID'], sort=False)
    .sum()
    .astype(int)
    .reset_index(name='n_illogical_routes')
)

rcp_illogical_counts

In [ ]:
multiple_illogical_routes_receptacles = rcp_illogical_counts[rcp_illogical_counts['n_illogical_routes'] > 1]['RECPTCL_FID'].unique()
print (len(multiple_illogical_routes_receptacles))
slightly_receptacles_df = receptacles_df_copy[~receptacles_df_copy['RECPTCL_FID'].isin(multiple_illogical_routes_receptacles)]
slightly_receptacles_df['RECPTCL_FID'].nunique()

In [ ]:
# Verify sums
print(f"Total receptacles: {receptacles_df_copy['RECPTCL_FID'].nunique()}")
print(f"Clean (0 illogical): {clean_receptacles_df['RECPTCL_FID'].nunique()}")
print(f"Slightly (0 or 1 illogical): {slightly_receptacles_df['RECPTCL_FID'].nunique()}")
print(f"More than 1 illogical: {len(multiple_illogical_routes_receptacles)}")
print(f"\nSlightly + Multiple = {slightly_receptacles_df['RECPTCL_FID'].nunique() + len(multiple_illogical_routes_receptacles)}")
print(f"Should equal total: {receptacles_df_copy['RECPTCL_FID'].nunique()}")

In [ ]:
clean_receptacles_df.to_csv('../data/interim/clean_receptacles_df.csv')
slightly_receptacles_df.to_csv('../data/interim/slightly_receptacles_df.csv')
receptacles_df_copy.to_csv('../data/interim/chaotic_receptacles_df.csv')